In [2]:
from llama_cpp import Llama
import json


llm = Llama(
    model_path="model/unsloth.Q4_K_M.gguf",
    n_ctx=4096,
    n_threads=8,          
    n_gpu_layers=-1,      
    n_batch=512           
)


llama_model_loader: loaded meta data with 29 key-value pairs and 147 tensors from model/unsloth.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Q4Km
llama_model_loader: - kv   3:                         general.size_label str              = 1.2B


llama_model_loader: - kv   4:                          llama.block_count u32              = 16
llama_model_loader: - kv   5:                       llama.context_length u32              = 131072
llama_model_loader: - kv   6:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   7:                  llama.feed_forward_length u32              = 8192
llama_model_loader: - kv   8:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   9:              llama.attention.head_count_kv u32              = 8
llama_model_loader: - kv  10:                       llama.rope.freq_base f32              = 500000.000000
llama_model_loader: - kv  11:     llama.attention.layer_norm_rms_epsilon f32              = 0.000010
llama_model_loader: - kv  12:                 llama.attention.key_length u32              = 64
llama_model_loader: - kv  13:               llama.attention.value_length u32              = 64
llama_model_loader: - kv  

In [11]:
list_of_strategies = [
    'ปิดฟ้าข้ามทะเล', 'ล้อมเวยช่วยจ้าว', 'ยืมดาบฆ่าคน', 'รอซ้ำยามเปลี้ย',
    'ตีชิงตามไฟ', 'ส่งเสียงบูรพาฝ่าตีประจิม', 'มีในไม่มี', 'ลอบตีเฉินชาง',
    'ดูไฟชายฝั่ง', 'ซ่อนดาบในรอยยิ้ม', 'หลี่ตายแทนถาว', 'จูงแพะติดมือ',
    'ตีหญ้าให้งูตื่น', 'ยืมซากคืนชีพ', 'ล่อเสือออกจากถ้ำ', 'แสร้งปล่อยเพื่อจับ',
    'โยนกระเบื้องล่อหยก', 'จับโจรเอาหัวโจก', 'ถอนฟืนใต้กระทะ', 'กวนน้ำจับปลา',
    'จักจั่นลอกคราบ', 'ปิดประตูจับโจร', 'คบไกลตีใกล้', 'ยืมทางพรางกล',
    'ลักขื่อเปลี่ยนเสา', 'ชี้ต้นหม่อนด่าต้นไหว', 'แสร้งทำบอแต่ไม่บ้า', 'ขึ้นบ้านชักบันได',
    'ต้นไม้ผลิดอก', 'สลับแขกเป็นเจ้าบ้าน', 'สาวงาม', 'เปิดเมือง', 'ไส้ศึก', 'ทุกข์กาย', 'ลูกโซ่', 'หลบหนี'
]

strategies_text = ", ".join(list_of_strategies)

input_data = "[1] P1: (0,3)→(4,3)\n[3] P1: (0,6)→(3,6)\n[5] P1: (0,1)→(2,1)"

# **ใช้สรุปกติกาเกมที่กระชับ**
game_rules_summary = """กฎกติกาเกมหมากหนีบ:
* **วัตถุประสงค์:** ผู้เล่นแต่ละฝ่ายมีหมาก 8 ตัวบนกระดาน 8x8 วัตถุประสงค์คือการ "หนีบ" กินหมากฝ่ายตรงข้ามให้เหลือน้อยที่สุดหรือจนหมดเพื่อชนะ
* **การเดินหมาก:** เดินได้ครั้งละ 1 ตัว ในแนวตรง (ขึ้น-ลง-ซ้าย-ขวา) กี่ช่องก็ได้ ห้ามเดินเฉียงหรือข้ามหมาก
* **การหนีบ (จับกิน):**
    * **รูปแบบ 1:** ใช้หมากของเรา 1 ตัวอยู่ตรงกลางระหว่างหมากฝ่ายตรงข้าม 2 ตัวเพื่อกินหมาก 2 ตัวนั้น
    * **รูปแบบ 2:** ใช้หมากของเรา 2 ตัว "หนีบ" ล้อมรอบหมากฝ่ายตรงข้าม 1-2 ตัวเพื่อกินหมากที่ถูกหนีบ
* **เงื่อนไขการชนะ:** กินหมากฝ่ายตรงข้ามหมด หรือทำให้คู่แข่งเดินหมากไม่ได้ หรือมีหมากเหลือน้อยที่สุดเมื่อจบเกม หากเหลือหมากเท่ากันถือว่าเสมอ
"""

# ปรับปรุง Instruction โดยรวม game_rules_summary, input_data และรูปแบบคำตอบเข้าด้วยกัน
instruction_with_input = f"""{game_rules_summary}

จากท่าเดินหมากที่กำหนดให้ จงวิเคราะห์และเลือกกลยุทธ์สามก๊กที่ตรงที่สุดจากรายการต่อไปนี้เท่านั้น:

{strategies_text}

ท่าเดินหมาก:
{input_data}

จงตอบในรูปแบบ:
กลยุทธ์: [ชื่อกลยุทธ์ที่คุณเลือก]
เหตุผล: [สรุปสั้นๆว่าท่าเดินหมากสอดคล้องกับกลยุทธ์ที่เลือกอย่างไร]"""

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:

### Response:
{}"""

full_prompt = alpaca_prompt.format(instruction_with_input, "")

from typing import cast, Dict, Any

response = cast(Dict[str, Any], llm(
    prompt=full_prompt,
    max_tokens=130,
    stop=["###", "\n\n", '<', '>', '-', '\n\n\n'],
    echo=False,
    temperature=0.7,
    stream=False
))

print(response['choices'][0]['text'].strip())


Llama.generate: 390 prefix-match hit, remaining 443 prompt tokens to eval
llama_perf_context_print:        load time =   46475.40 ms
llama_perf_context_print: prompt eval time =    5225.66 ms /   443 tokens (   11.80 ms per token,    84.77 tokens per second)
llama_perf_context_print:        eval time =    7819.35 ms /   129 runs   (   60.62 ms per token,    16.50 tokens per second)
llama_perf_context_print:       total time =   13285.98 ms /   572 tokens


กลยุทธ์: สอดคล้องกับกลยุทธ์: ตีหญ้าให้งูตื่น
เหตุผล: ตีหญ้าให้งูตื่นเป็นกลยุทธ์ที่คุณเลือก เนื่องจากคุณจำเป็นต้องสร้างความวุ่นวายให้กับผู้เล่นตรงข้าม โดยคุณขยับหมากที่อยู่ในแนวที่ดูเหมาะสม (ในกรณีนี้คือจุด 3) ไปยังแนวที่เป็นไปไม่ได้ (ในกรณีนี้คือจุด 1) จากน


In [10]:
list_of_strategies = [
    'ปิดฟ้าข้ามทะเล', 'ล้อมเวยช่วยจ้าว', 'ยืมดาบฆ่าคน', 'รอซ้ำยามเปลี้ย',
    'ตีชิงตามไฟ', 'ส่งเสียงบูรพาฝ่าตีประจิม', 'มีในไม่มี', 'ลอบตีเฉินชาง',
    'ดูไฟชายฝั่ง', 'ซ่อนดาบในรอยยิ้ม', 'หลี่ตายแทนถาว', 'จูงแพะติดมือ',
    'ตีหญ้าให้งูตื่น', 'ยืมซากคืนชีพ', 'ล่อเสือออกจากถ้ำ', 'แสร้งปล่อยเพื่อจับ',
    'โยนกระเบื้องล่อหยก', 'จับโจรเอาหัวโจก', 'ถอนฟืนใต้กระทะ', 'กวนน้ำจับปลา',
    'จักจั่นลอกคราบ', 'ปิดประตูจับโจร', 'คบไกลตีใกล้', 'ยืมทางพรางกล',
    'ลักขื่อเปลี่ยนเสา', 'ชี้ต้นหม่อนด่าต้นไหว', 'แสร้งทำบอแต่ไม่บ้า', 'ขึ้นบ้านชักบันได',
    'ต้นไม้ผลิดอก', 'สลับแขกเป็นเจ้าบ้าน', 'สาวงาม', 'เปิดเมือง', 'ไส้ศึก', 'ทุกข์กาย', 'ลูกโซ่', 'หลบหนี'
]

# ใส่ action_id ลงในรายการกลยุทธ์
strategies_text = ", ".join([f"{i}: {s}" for i, s in enumerate(list_of_strategies)])

input_data = "[1] P1: (0,3)→(4,3)\n[3] P1: (0,6)→(3,6)\n[5] P1: (0,1)→(2,1)"

game_rules_summary = """กฎกติกาเกมหมากหนีบ:
* **วัตถุประสงค์:** ผู้เล่นแต่ละฝ่ายมีหมาก 8 ตัวบนกระดาน 8x8 วัตถุประสงค์คือการ "หนีบ" กินหมากฝ่ายตรงข้ามให้เหลือน้อยที่สุดหรือจนหมดเพื่อชนะ
* **การเดินหมาก:** เดินได้ครั้งละ 1 ตัว ในแนวตรง (ขึ้น-ลง-ซ้าย-ขวา) กี่ช่องก็ได้ ห้ามเดินเฉียงหรือข้ามหมาก
* **การหนีบ (จับกิน):**
    * **รูปแบบ 1:** ใช้หมากของเรา 1 ตัวอยู่ตรงกลางระหว่างหมากฝ่ายตรงข้าม 2 ตัวเพื่อกินหมาก 2 ตัวนั้น
    * **รูปแบบ 2:** ใช้หมากของเรา 2 ตัว "หนีบ" ล้อมรอบหมากฝ่ายตรงข้าม 1-2 ตัวเพื่อกินหมากที่ถูกหนีบ
* **เงื่อนไขการชนะ:** กินหมากฝ่ายตรงข้ามหมด หรือทำให้คู่แข่งเดินหมากไม่ได้ หรือมีหมากเหลือน้อยที่สุดเมื่อจบเกม หากเหลือหมากเท่ากันถือว่าเสมอ
"""

# รวมคำสั่งและอินพุต
instruction_with_input = f"""{game_rules_summary}

จากท่าเดินหมากที่กำหนดให้ จงวิเคราะห์และเลือกกลยุทธ์สามก๊กที่ตรงที่สุดจากรายการต่อไปนี้เท่านั้น พร้อมระบุลำดับ action_id ของกลยุทธ์นั้น (เริ่มนับจาก 0):

{strategies_text}

ท่าเดินหมาก:
{input_data}

จงตอบในรูปแบบ:
กลยุทธ์: [ชื่อกลยุทธ์ที่คุณเลือก]
action_id: [เลขลำดับของกลยุทธ์ในรายการ เริ่มจาก 0]
"""

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:

### Response:
{}"""

# รวมเข้าเป็น prompt เดียว
full_prompt = alpaca_prompt.format(instruction_with_input, "")

# เรียกใช้โมเดล (สมมุติว่า llm คือฟังก์ชันโหลดโมเดลของคุณไว้แล้ว)
from typing import cast, Dict, Any

response = cast(Dict[str, Any], llm(
    prompt=full_prompt,
    max_tokens=130,
    stop=["###", "\n\n", '<', '>', '-', '\n\n\n'],
    echo=False,
    temperature=0.7,
    stream=False
))

print(response['choices'][0]['text'].strip())


Llama.generate: 951 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =   46475.40 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =   20003.37 ms /   130 runs   (  153.87 ms per token,     6.50 tokens per second)
llama_perf_context_print:       total time =   20902.55 ms /   131 tokens


คุณเป็นเจ้าบ้านของสนามหมาก 8 ตัวที่อยู่ในแนวตั้งแต่ทางข้างของศัตรูจนถึงแนวตั้งด้วย ดังนั้น ตัวเลข 1 ของคุณเป็น 1 ตา จากนั้นอยู่ทางข้ามของศัตรูที่อยู่ทางข้างของศัตรูจนถึงแนวตั้งด้วย ดังนั้น ตัวเลข 2 ของคุณเป็น 2 ตา จากนั้นอยู่ที่ด้านข้างของศัตรูที่อยู่ทาง
